# Joining Weather and flight data

In this jupyter notebook i joined the weather and flight data.

I also did feature engineering to create and derive new features

I also filled nan values

## Importing Libraries

In [35]:
import os
import numpy as np
import pandas as pd

## Get the weather data into a single dataframe

This is done so that later on it is easy to merge with the flight data

In [36]:
def get_weather_df(folder_name = 'Weather_formatted'):
    weather_folder = folder_name
    weather_files = [f for f in os.listdir(weather_folder) if f.endswith('.csv')]
    weather_data = []
    for file in weather_files:
        month_name = file.split('.')[0]
        weather_df = pd.read_csv(os.path.join(weather_folder, file))
        weather_df['Month'] = month_name
        weather_data.append(weather_df)
    weather_df_combined = pd.concat(weather_data, ignore_index=True)
    weather_df_combined['Day'] = weather_df_combined['Day'].astype(int)
    weather_df_combined.to_csv('combined_weather.csv')
    return weather_df_combined

## Combine weather and flight data

In [37]:
def merge_dfs(main_df,weather_df):
    result_df = pd.merge(
        main_df,
        weather_df,
        on=['Month', 'Day'],
        how='left'
    )
    return result_df

In [38]:
def combine_weather_and_flight_data(main_csv_path,weather_folder='Weather_formatted'):
    flight_data = pd.read_csv(main_csv_path)
    needed_features = ['flight_number','airline_name','code_shared_flag','type','status','departure_iata_code','departure_icao_code','departure_scheduled_time',
                    'departure_actual_time','arrival_iata_code','arrival_icao_code','arrival_estimated_time']
    filtered_flight_data = flight_data[needed_features]
    columns_to_convert_to_datetime = ['departure_scheduled_time','departure_actual_time','arrival_estimated_time']
    for col in columns_to_convert_to_datetime:
        filtered_flight_data[col] = pd.to_datetime(filtered_flight_data[col], errors='coerce')
    df = filtered_flight_data.copy()
    df['Month'] = df['departure_scheduled_time'].dt.strftime('%b')
    df['Day'] = df['departure_scheduled_time'].dt.day
    weather_df = get_weather_df(weather_folder)
    result_df = merge_dfs(df,weather_df=weather_df)
    return result_df

In [39]:
df = combine_weather_and_flight_data('output.csv')

/tmp/ipykernel_8406/347225113.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_flight_data[col] = pd.to_datetime(filtered_flight_data[col], errors='coerce')
/tmp/ipykernel_8406/347225113.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_flight_data[col] = pd.to_datetime(filtered_flight_data[col], errors='coerce')
/tmp/ipykernel_8406/347225113.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

## Feature Engineering

Tasks Left:

Delay time in minutes banana

Day of the week banana

Hour of the day

Month(already done)

### Calculate Delay time

In [ ]:
df['departure_scheduled_time'] = pd.to_datetime(df['departure_scheduled_time'])
df['departure_actual_time'] = pd.to_datetime(df['departure_actual_time'], errors='coerce')
df['delay_time'] = (df['departure_actual_time'] - df['departure_scheduled_time']).dt.total_seconds()
df['delay_time'] = df['delay_time'] / 60

In [ ]:
x = ['departure_scheduled_time','departure_actual_time','delay_time']
df[x].head()

,arrival_scheduled_time,arrival_estimated_time,delay_time
0,2024-01-31 13:00:00,2024-01-31 13:16:00,-16.0
1,2024-01-25 15:05:00,NaT,NaN
2,2024-01-27 15:40:00,2024-01-27 15:41:00,-1.0
3,2024-01-19 15:05:00,NaT,NaN
4,2024-01-25 02:55:00,2024-01-25 03:06:00,-11.0


### Find day of the week

In [8]:
df['day_of_week'] = df['departure_scheduled_time'].dt.day_name()

In [9]:
df[['day_of_week','departure_scheduled_time']].head()

,day_of_week,departure_scheduled_time
0,Wednesday,2024-01-31 13:00:00
1,Thursday,2024-01-25 15:05:00
2,Saturday,2024-01-27 15:40:00
3,Friday,2024-01-19 15:05:00
4,Thursday,2024-01-25 02:55:00


In [10]:
df[['day_of_week','departure_scheduled_time']].isnull().sum()

day_of_week                 0
departure_scheduled_time    0
dtype: int64

### Find hour of the day

In [11]:
df['hour_of_day'] = df['departure_scheduled_time'].dt.hour

In [12]:
df[['hour_of_day','departure_scheduled_time']].head()

,hour_of_day,departure_scheduled_time
0,13,2024-01-31 13:00:00
1,15,2024-01-25 15:05:00
2,15,2024-01-27 15:40:00
3,15,2024-01-19 15:05:00
4,2,2024-01-25 02:55:00


In [13]:
df[['hour_of_day','departure_scheduled_time']].isnull().sum()

hour_of_day                 0
departure_scheduled_time    0
dtype: int64

In [14]:
df[['delay_time','Month','day_of_week','hour_of_day','departure_scheduled_time','departure_actual_time']].head()

,delay_time,Month,day_of_week,hour_of_day,departure_scheduled_time,departure_actual_time
0,16.0,Jan,Wednesday,13,2024-01-31 13:00:00,2024-01-31 13:16:00
1,NaN,Jan,Thursday,15,2024-01-25 15:05:00,NaT
2,1.0,Jan,Saturday,15,2024-01-27 15:40:00,2024-01-27 15:41:00
3,NaN,Jan,Friday,15,2024-01-19 15:05:00,NaT
4,11.0,Jan,Thursday,2,2024-01-25 02:55:00,2024-01-25 03:06:00


## Filling nan values of delay_time column (skipped for now)

In [15]:
df.isnull().sum()

flight_number                   0
airline_name                    0
code_shared_flag                0
type                            0
status                          0
departure_iata_code             0
departure_icao_code             0
departure_scheduled_time        0
departure_actual_time       20582
arrival_iata_code               0
arrival_icao_code               0
arrival_estimated_time      21364
Month                           0
Day                             0
Temperature (°F)_max            0
Temperature (°F)_avg            0
Temperature (°F)_min            0
Dew Point (°F)_max              0
Dew Point (°F)_avg              0
Dew Point (°F)_min              0
Humidity (%)_max                0
Humidity (%)_avg                0
Humidity (%)_min                0
Wind Speed (mph)_max            0
Wind Speed (mph)_avg            0
Wind Speed (mph)_min            0
Pressure (in)_max               0
Pressure (in)_avg               0
Pressure (in)_min               0
delay_time    

In [39]:
df.loc[df['status'] == 'active', 'delay_time'] = df.loc[df['status'] == 'active', 'delay_time'].fillna(
    df.loc[df['status'] == 'active', 'delay_time'].mean()
)

In [41]:
df.isnull().sum()

flight_number                   0
type                            0
status                          0
departure_iata_code             0
departure_icao_code             0
departure_scheduled_time        0
departure_actual_time       20582
arrival_iata_code               0
arrival_icao_code               0
arrival_estimated_time      21364
Month                           0
Day                             0
Temperature (°F)_max            0
Temperature (°F)_avg            0
Temperature (°F)_min            0
Dew Point (°F)_max              0
Dew Point (°F)_avg              0
Dew Point (°F)_min              0
Humidity (%)_max                0
Humidity (%)_avg                0
Humidity (%)_min                0
Wind Speed (mph)_max            0
Wind Speed (mph)_avg            0
Wind Speed (mph)_min            0
Pressure (in)_max               0
Pressure (in)_avg               0
Pressure (in)_min               0
delay_time                   3664
day_of_week                     0
hour_of_day   

## Saving to my train.csv

In [16]:
df.to_csv('train.csv',index=False)